In [2]:
import pandas as pd
import numpy as np
import sys
import os

# Allows to get the module in utils
sys.path.append(os.path.abspath(".."))


from utils.load import load_data
from utils.preprocessing import preprocess
from utils.split import split_data

from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
# from sklearn.compose import ColumnTransformer

import tensorflow as tf

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import class_weight

import datetime

(4507, 20) (1409, 20) (1127, 20) (1409, 20) (1127,) (1409,)


2025-05-21 09:18:52.479431: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-21 09:18:52.482111: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-21 09:18:52.491107: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747811932.506736  114432 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747811932.511357  114432 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747811932.521841  114432 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [3]:
# Load data
df = load_data()

In [4]:
# Split and preprocess datas set
X_train, X_val, X_test, y_train, y_val, y_test = split_data(df)

X_train_processed, X_test_processed, X_val_processed, y_test_encoding, y_train_encoded, y_val_encoding, pipeline, le = preprocess(X_train, X_val, X_test, y_train, y_val, y_test)

/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [5]:
X_train_dense = X_train_processed.toarray() if hasattr(X_train_processed, "toarray") else X_train_processed
X_val_dense = X_val_processed.toarray() if hasattr(X_val_processed, "toarray") else X_val_processed
X_test_dense = X_test_processed.toarray() if hasattr(X_test_processed, "toarray") else X_test_processed

In [6]:
y_train_vector = y_train_encoded.reshape(-1)
y_val_vector = y_val_encoding.reshape(-1)
y_test_vector = y_test_encoding.reshape(-1)

In [7]:
input_dim = X_train_dense.shape[1]

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(input_dim,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss='binary_crossentropy',
    metrics=['accuracy', 'recall', 'auc']
)

model.summary()

E0000 00:00:1747811934.252913  114432 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1747811934.253266  114432 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │       290,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 294,593 (1.12 MB)

 Trainable params: 294,593 (1.12 MB)

 Non-trainable params: 0 (0.00 B)

### Exemple of checkpoints use

model.compile(loss=..., optimizer=...,
              metrics=['accuracy'])

EPOCHS = 10
checkpoint_filepath = '/tmp/ckpt/checkpoint.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

- Model is saved at the end of every epoch, if it's the best seen so far.
model.fit(epochs=EPOCHS, callbacks=[model_checkpoint_callback])

- The model (that are considered the best) can be loaded as -
keras.models.load_model(checkpoint_filepath)

- Alternatively, one could checkpoint just the model weights as -
checkpoint_filepath = '/tmp/ckpt/checkpoint.weights.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

- Model weights are saved at the end of every epoch, if it's the best seen so far.
model.fit(epochs=EPOCHS, callbacks=[model_checkpoint_callback])

- The model weights (that are considered the best) can be loaded as -
model.load_weights(checkpoint_filepath)


In [9]:
checkpoint_path = "checkpoints/churn_models.keras"
os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',
    mode='max',  
    save_best_only=True,# sauvegarde le modèle qui maximise l’accuracy de validation
    verbose=1
)

In [10]:
history = model.fit(
    X_train_dense, y_train_vector,
    validation_data=(X_val_dense, y_val_vector),
    epochs=20,
    batch_size=16,
    callbacks=[model_ckpt],
    verbose=1
)

Epoch 1/20
271/282 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7692 - auc: 0.7929 - loss: 0.4682 - recall: 0.3909
Epoch 1: val_accuracy improved from -inf to 0.80568, saving model to checkpoints/churn_models.keras
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7698 - auc: 0.7939 - loss: 0.4675 - recall: 0.3940 - val_accuracy: 0.8057 - val_auc: 0.8417 - val_loss: 0.4296 - val_recall: 0.5084
Epoch 2/20
280/282 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8494 - auc: 0.9012 - loss: 0.3433 - recall: 0.6409
Epoch 2: val_accuracy did not improve from 0.80568
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8494 - auc: 0.9012 - loss: 0.3433 - recall: 0.6409 - val_accuracy: 0.7782 - val_auc: 0.8223 - val_loss: 0.4581 - val_recall: 0.3311
Epoch 3/20
279/282 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9708 - auc: 0.9940 - loss: 0.0819 - recall: 0.9140
Epoch 3: val_accuracy did not improve from 0.80568
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9710 - auc: 0.9940

In [11]:
y_pred_probs = model.predict(X_test_dense)
y_pred = np.argmax(y_pred_probs, axis=1)
print("\nClassification Report :")
print(classification_report(y_test_vector, y_pred))

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

Classification Report :
              precision    recall  f1-score   support

           0       0.73      1.00      0.85      1035
           1       0.00      0.00      0.00       374

    accuracy                           0.73      1409
   macro avg       0.37      0.50      0.42      1409
weighted avg       0.54      0.73      0.62      1409



/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

In [12]:
print("Matrice de Confusion :")
cm = confusion_matrix(y_test_vector, y_pred)
print(cm)

Matrice de Confusion :
[[1035    0]
 [ 374    0]]


In [13]:
y_pred_probs = model.predict(X_test_dense)

probs = y_pred_probs.ravel()

y_pred = (probs >= 0.5)
print("\nClassification Report :")
print(classification_report(y_test_vector, y_pred))
y_test_vector.shape, y_pred_probs.shape

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 

Classification Report :
              precision    recall  f1-score   support

           0       0.80      0.89      0.84      1035
           1       0.55      0.37      0.44       374

    accuracy                           0.75      1409
   macro avg       0.67      0.63      0.64      1409
weighted avg       0.73      0.75      0.73      1409



((1409,), (1409, 1))

In [14]:
%load_ext tensorboard

In [15]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',    # surveille la perte de validation
    patience=3,            # tolère 3 époques sans amélioration
    restore_best_weights=True
)

In [16]:
log_dir = os.path.join(
    "logs", "fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
)
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,       # enregistre les histogrammes de poids chaque époque
    write_graph=True,       # sauvegarde le graph du modèle
    write_images=True
)

In [17]:
history = model.fit(
    X_train_dense, y_train_vector,
    validation_split=0.2,
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop, model_ckpt, tensorboard_cb]
)

Epoch 1/20
221/226 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - auc: 1.0000 - loss: 6.7295e-07 - recall: 1.0000
Epoch 1: val_accuracy improved from 0.80568 to 1.00000, saving model to checkpoints/churn_models.keras
226/226 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 1.0000 - auc: 1.0000 - loss: 6.7224e-07 - recall: 1.0000 - val_accuracy: 1.0000 - val_auc: 1.0000 - val_loss: 5.4797e-07 - val_recall: 1.0000
Epoch 2/20
222/226 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - auc: 1.0000 - loss: 5.4067e-07 - recall: 1.0000
Epoch 2: val_accuracy did not improve from 1.00000
226/226 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 1.0000 - auc: 1.0000 - loss: 5.4106e-07 - recall: 1.0000 - val_accuracy: 1.0000 - val_auc: 1.0000 - val_loss: 4.9092e-07 - val_recall: 1.0000
Epoch 3/20
214/226 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - auc: 1.0000 - loss: 4.6965e-07 - recall: 1.0000
Epoch 3: val_accuracy did not improve from 1.00000
226/226 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step -

In [18]:
history = model.fit(
    X_train_dense, y_train_vector,
    validation_data=(X_val_dense, y_val_vector),
    epochs=20,
    batch_size=16,
    callbacks=[model_ckpt,tensorboard_cb],
    verbose=1
)

Epoch 1/20
274/282 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - auc: 1.0000 - loss: 3.8734e-08 - recall: 1.0000
Epoch 1: val_accuracy did not improve from 1.00000
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 1.0000 - auc: 1.0000 - loss: 3.8735e-08 - recall: 1.0000 - val_accuracy: 0.7320 - val_auc: 0.6930 - val_loss: 0.8918 - val_recall: 0.2642
Epoch 2/20
280/282 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - auc: 1.0000 - loss: 3.4663e-08 - recall: 1.0000
Epoch 2: val_accuracy did not improve from 1.00000
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 1.0000 - auc: 1.0000 - loss: 3.4639e-08 - recall: 1.0000 - val_accuracy: 0.7329 - val_auc: 0.6929 - val_loss: 0.9022 - val_recall: 0.2575
Epoch 3/20
271/282 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - auc: 1.0000 - loss: 2.7525e-08 - recall: 1.0000
Epoch 3: val_accuracy did not improve from 1.00000
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 1.0000 - auc: 1.0000 - loss: 2.7517e-08 - recall:

In [19]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 105293), started 16:52:35 ago. (Use '!kill 105293' to kill it.)

In [22]:
print(y_train_vector)

[0 0 0 ... 0 0 0]


In [24]:
x_train=0
for i in y_train_vector:
    if i == 1:
        x_train+=1
x_val=0
for i in y_val_vector:
    if i == 1:
        x_val+=1

x_test=0
for i in y_test_vector:
    if i == 1:
        x_test+=1

print(x_train, x_val, x_test)


1196 299 374


In [30]:
y_val_vector.shape

(1127,)